# <center><font color='#333333'>Use Python to Stylize the Excel Formatting</font></center>
### <center><font color='#3b5998'>Created by cyda - Yeung Wong & Carrie Lo</font></center>

--------------------------------------------------------------------------------------
![logo](https://4.bp.blogspot.com/-LAXjdvVCYCU/WxeQFKQ-1wI/AAAAAAAAACs/o8IJ1eLLAEwQYv2Az7EqQi9jODTqRx7wACK4BGAYYCw/s1000/tight%2Bbanner_with_description.png)

--------------------------------------------------------------------------------------
Please acknowledge <b>team cyda - Yeung Wong & Carrie Lo</b> when using the code

<b><font color='#3b5998'>If you find this script is helpful, please feel free to endorse us through Linkedin!</font></b>

<b>Linkedin:</b>

Yeung Wong - https://www.linkedin.com/in/yeungwong/

Carrie Lo - https://www.linkedin.com/in/carrielsc/

--------------------------------------------------------------------------------------

# Step 1 - Data Scraping

In [1]:
import json
from urllib import request
url = "https://production.api.coindesk.com/v2/tb/price/ticker?assets=all"
response = request.urlopen(url)
data = json.loads(response.read().decode())
data

{'statusCode': 200,
 'message': 'OK',
 'data': {'BTC': {'iso': 'BTC',
   'name': 'Bitcoin',
   'slug': 'bitcoin',
   'change': {'percent': 2.713846028057471, 'value': 1207.072774},
   'ohlc': {'o': 44478.307226,
    'h': 46843.67,
    'l': 43480.540171,
    'c': 45685.38},
   'circulatingSupply': 18814943.79249204,
   'marketCap': 859567856838.6399,
   'ts': 1631606099000,
   'src': 'tb'},
  'ETH': {'iso': 'ETH',
   'name': 'Ethereum',
   'slug': 'ethereum',
   'change': {'percent': 2.996461607015777, 'value': 96.209888},
   'ohlc': {'o': 3210.783271, 'h': 3385, 'l': 3116.81, 'c': 3306.993159},
   'circulatingSupply': 117224668.1714915,
   'marketCap': 387661175709.1675,
   'ts': 1631606099000,
   'src': 'tb'},
  'XRP': {'iso': 'XRP',
   'name': 'XRP',
   'slug': 'xrp',
   'change': {'percent': 2.7825691560965105, 'value': 0.0291316994},
   'ohlc': {'o': 1.0469353229,
    'h': 1.0996056267,
    'l': 1.0266491573,
    'c': 1.0762684335},
   'circulatingSupply': 99990179359.64815,
   'ma

# Step 2 - Data Tabulation

In [2]:
def flatten_json(coin_dict):
    flatten_dict = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for key in x:
                flatten(x[key], name + key + '_')
        else:
            flatten_dict[name[:-1]] = x
    flatten(coin_dict)
    return flatten_dict

import pandas as pd
master_df = pd.DataFrame()
for coin in data['data'].keys():
    temp_df = pd.json_normalize(flatten_json(data['data'][coin]))
    master_df = master_df.append(temp_df)
master_df = master_df[['iso', 'name', 'ohlc_o', 'ohlc_h', 'ohlc_l', 'ohlc_c', 'change_percent']].reset_index(drop=True)
master_df.columns = ['Symbol', 'Name', 'Open', 'High', 'Low', 'Close', 'Pct_Change']
master_df.iloc[:, 2:] = master_df.iloc[:, 2:].apply(lambda x: round(x, 2))
master_df['Pct_Change'] = master_df['Pct_Change'] / 100
master_df = master_df.sort_values('Pct_Change', ascending=False).reset_index(drop=True)
master_df.to_csv('master_df.csv', index=False)
master_df.head()

,Symbol,Name,Open,High,Low,Close,Pct_Change
0,ICX,ICON,1.61,1.99,1.52,1.88,0.1698
1,ATOM,Cosmos,30.95,38.95,30.31,36.04,0.1622
2,XTZ,Tezos,6.87,8.02,6.31,7.78,0.1339
3,FET,Fetch.ai,0.81,1.01,0.79,0.91,0.1285
4,TRX,Tron,0.10,0.11,0.10,0.11,0.0834


# Step 3 - Excel Formatting

In [3]:
file_name = "Cryptocurrency.xlsx"
sheet_name = "Summary"

writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
master_df.to_excel(writer, sheet_name=sheet_name, startrow = 2, index = False)

from datetime import datetime
workbook  = writer.book
worksheet = writer.sheets[sheet_name]

worksheet.write(0, 0, 'Cryptocurrency Pricing Summary on '+datetime.now().strftime('%d %b %Y'), workbook.add_format({'bold': True, 'color': '#E26B0A', 'size': 14}))

worksheet.write(len(master_df)+4, 0, 'Remark:', workbook.add_format({'bold': True}))
worksheet.write(len(master_df)+5, 0, 'The last update time is ' + datetime.now().strftime('%H:%M') + '.')

header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'fg_color': '#FDE9D9', 'border': 1})
for col_num, value in enumerate(master_df.columns.values):
    worksheet.write(2, col_num, value, header_format)

row_idx, col_idx = master_df.shape
for r in range(row_idx):
    for c in range(col_idx):
        if c == 6:
            worksheet.write(r + 3, c, master_df.values[r, c], workbook.add_format({'border': 1, 'num_format': '0.00%'}))
        else:
            worksheet.write(r + 3, c, master_df.values[r, c], workbook.add_format({'border': 1, 'num_format': '0.00'}))

worksheet.set_column(0, 6, 12)
worksheet.set_column(1, 1, 20)
    
writer.save()